# Simple Tests on the Queen's Flume Experiments

## Non-equilibrium Model

This page is updated from the 'http://127.0.0.1:8888/notebooks/Queens%20Flume%20Experiement%20-%20August%2015%202016.ipynb' 

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import cython
%load_ext cython
%matplotlib inline
%load_ext autoreload
%autoreload 2



In [6]:
%%cython
import sys
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
from scipy.interpolate import interp1d
from copy import copy, deepcopy

import collections

sys.path.append("/home/pgrover/Dropbox/PhD/Morphodynamic_Model/pymorph")
from schemes.weno import get_left_flux,get_right_flux
from schemes.weno import get_stencil
import sediment_transport.sed_trans as sedtrans
from models.shallow_water_solver import shallow_solver
from schemes.avalanche_scheme import *

#filepath = '/home/pgrover/working/working/Paper_4_Turbulence_Modelling/FlumeExperiment/geometries/flume_profile_smoothed.txt'
filepath = '/home/pgrover/Paper_4_Turbulence_Modelling/FlumeExperiment/geometries/flume_profile_smoothed_v002.txt'

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
retval = collections.OrderedDict()


z = []
with open(filepath) as f:
    for line in f:
        values = line.split()
        if is_number(values[0])==False:
            if float(values[1]).is_integer():
                retval[values[0]]=int(values[1])
            else:
                retval[values[0]]=float(values[1])
        else:
            z.append(float(values[0]))
z = np.array(z)
print('Z: {0}'.format(len(z)))
xmax = float(retval['nrows']) * retval['cellsize']
nx = retval['nrows']
dx = retval['cellsize']

#--------------------------------
# Increase the resolution on the grid
#--------------------------------
resolution = 2
x = np.linspace(0, nx*dx, num=len(z))
f = interp1d(x, z)
xnew = np.linspace(0, nx*dx, num=len(z)*resolution)
znew = f(xnew)
nx = len(xnew)

print('Finished reinterpolating the grid nx={0}'.format(len(xnew)))

#--------------------------------
# Set up the constants
#--------------------------------
a=0.001
b=3.0
Q=10.0
nP=0.4
Lb = 1.1    # Adaptation length 
s= 0.2      # Surface water elevation
beta = 0.01

# Sediment properties
D50 = 0.001     # Particle Diameter
rho_particule = 2650

#--------------------------------
# Set up the domain, BCs and ICs
#--------------------------------
import logging
logger = logging.getLogger('pyclaw')
logger.setLevel(logging.CRITICAL)

x = pyclaw.Dimension(0.0,xmax,len(xnew),name='x')
domain = pyclaw.Domain(x)
state = pyclaw.State(domain,1)
xc = state.grid.x.centers
dx = state.grid.delta[0]
print('Grid dx = {0}'.format(dx))
print('Grid nx = {0}'.format(len(xc)))

zc = znew.copy()
z_hydro = znew.copy()

 # Gravitational constant
state.problem_data['grav'] = 9.8
state.problem_data['sea_level'] = 0.0


# Set up the inital state of the flow
shw_slvr = shallow_solver(domain)
u, surf,h = shw_slvr.run(zc,s)


if np.isnan(u).any():
    # Get the flow
    print u
    raise ValueError('Failed to calculate the flow')


#--------------------------------
# Initialize the sed transport 
#--------------------------------    
# Eq 57
qbedload = np.zeros(nx)
qbedstar = np.zeros(nx)
roe_speed = np.zeros(nx)
slope = np.gradient(zc)
for i in range(0,nx): #i=2        
    zlocal = get_stencil(zc,i-2,i+2)
    # equilibrium bed load
    qbedstar[i] = sedtrans.get_unit_bed_load_slope(h[i],u[i],D50,slope[i], rho_particule, angleReposeDegrees = 30.0, type='mpm')
flux = np.zeros(nx)

#--------------------------------
#  Setup the model run parameters
#--------------------------------
simTime = 60. # Runtime in seconds
dt = 0.5 # Time step size (s)
nt = int(simTime/dt)  # Number of time steps


#--------------------------------
# Set up the model reporting parameters
#--------------------------------
z0 = zc.copy() 
z600 = zc.copy() 
z2000 = zc.copy()
verts = []
extractionInterval = 100.
tsteps=[]
iterations = 0
cntr=0
#--------------------------------
#  Run the model
#--------------------------------
for n in range(1,nt):  
    zn = zc.copy() 
    for i in range(0,nx): #i=2
        zloc = get_stencil(zn,i-2,i+4)        
        # Since k=3
        # stencil is i-2 to i+2 
        qloc = get_stencil(qbedload,i-2,i+4)
        if len(qloc)<>6:
            raise ValueError('Stencil is incorrect')
 
        # Determine the Upwind flux
        # The 0.5 comes from the c+abs(c) which is 2 if the wave speed is +ive
        # this is the evaluation of the left and right based fluxes. Eq. 18 and 19        
        if (zloc[3]-zloc[2]) == 0.0:
            roe_speed[i]=np.sign( (qloc[3]-qloc[2]) )
        else:
            roe_speed[i]=np.sign( (qloc[3]-qloc[2])/ (zloc[3]-zloc[2]) )
        
        if roe_speed[i] >= 0.0:
            flux[i] = get_left_flux(qloc)
        else:
            flux[i] = get_right_flux(qloc)
        
    # Need the sign of the phase speed
    # Need to check this out
    for i in range(0,nx): #i=2       
        floc = get_stencil(flux,i-1,i+1)
        zc[i] = zn[i]-(1./(1.-nP))*dt/dx*(floc[1]-floc[0])
    
    bed_max_delta = np.max(np.abs(zn - zc))
    
    # Apply the avalanche model
    zc,iterations1 = avalanche_model(dx,xc,zc)
    
    #zc = z_updated.copy()
    
    # Re-run the solver and updated the sediment transport
    # Check if the avalanche model was updated OR the bed changed greater than the threshold
    #if iterations > 10 or bed_max_delta > 0.00005:
    iterations+=1
    if iterations > 5:
        u, surf,h = shw_slvr.run(zc,s)
        iterations = 0
        #print('Updated')
        # Remove the old method 
        #qbedload = a*u**b
        slope = np.gradient(zc)

        for i in range(0,nx): #i=2        
            zlocal = get_stencil(zc,i-2,i+2)
            # equilibrium bed load
            qbedstar[i] = sedtrans.get_unit_bed_load_slope(h[i],u[i],D50,slope[i], 
                                                           rho_particule, angleReposeDegrees = 30.0, type='mpm')

    for iter in range(5):
        qbedloadnew = np.zeros(nx)
        dtLocal = 0.5
        for i in range(0,nx):
            qbedlocal = get_stencil(qbedload,i-2,i+2)            
            qbedloadnew[i] = qbedload[i] - 0.5 * (dtLocal/dx)*(qbedlocal[3]-qbedlocal[1]) + (dtLocal/Lb)*(qbedstar[i]-qbedlocal[2])
        resid = np.mean(qbedloadnew-qbedload)
        print(iter,resid)
        qbedload = deepcopy(qbedloadnew)            
        
        
        #qbedload[i] = sedtrans.get_unit_bed_load2(h[i],u[i],D50,rho_particule, type='mpm')
    
    if (n*dt / extractionInterval) == math.floor(n*dt / extractionInterval):
        verts.append(list(zip(xc.copy(),zc.copy())))
        tsteps.append(n*dt)        
        #np.save('flume_dunes.txt', verts)
    
    cntr += 1
    if cntr == 100:
        cntr = 0
        print('Time step {0} of {1} - bed_max_delta {2}'.format(n,nt,bed_max_delta))    
        
print('Done')

Z: 87
Finished reinterpolating the grid nx=174
Grid dx = 0.015
Grid nx = 174
(0, 2.3063866764614107e-07)
(1, 1.2580290962516791e-07)
(2, 6.8619768886455602e-08)
(3, 3.7428964847151465e-08)
(4, 2.0415799007850683e-08)


ValueError: f(a) and f(b) must have different signs

In [ ]:
fig = plt.figure(figsize=(12, 4))
plt.plot(xc,surf)
plt.plot(xc,znew,'g.')
plt.plot(xc,zc,'r')

In [ ]:
plt.plot(xc,u)
#plt.plot(xc,qbedload*1000000.)

In [ ]:
qb=np.zeros(nx)
for i in range(0,nx): #i=2        
    zlocal = get_stencil(zc,i-2,i+2)
    #qbedload[i] = sedtrans.get_unit_bed_load_slope(h[i],u[i],D50,slope, rho_particule, angleReposeDegrees = 30.0, type='mpm')
    qb[i] = sedtrans.get_unit_bed_load2(h[i],u[i],D50,rho_particule, type='mpm')

plt.plot(xc,qbedload*100000.,'b--')
plt.plot(xc,qb*100000.,'k')
plt.plot(xc,zc,'r')
plt.plot(xc,u)

In [ ]:
qb_noslope=np.zeros(nx)
qb_slope = np.zeros(nx)
for i in range(0,nx): #i=2        
    zlocal = get_stencil(zn,i-2,i+2)
    #slope = 0.0#(zlocal[3]-zlocal[1]) / dx
    qb_slope[i] = sedtrans.get_unit_bed_load_slope(h[i],u[i],D50,slope[i], rho_particule, angleReposeDegrees = 30.0, type='mpm')
    qb_noslope[i] = sedtrans.get_unit_bed_load2(h[i],u[i],D50,rho_particule, type='mpm')

In [ ]:
print(len(xc),len(qb))

In [ ]:
gradQbed_slope= np.gradient(qb_slope)
gradqbed_noslope = np.gradient(qb_noslope)

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.plot(xc,gradQbed_slope, label='With slope adjustment')
plt.plot(xc,gradqbed_noslope, label='No slope adjustment')
plt.plot(xc,zc/200000., label='bed')
plt.legend(loc=4)
plt.ylabel('Grad Bedload kg/s')
#plt.plot(xc,slope/10000)

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.plot(xc,qb_slope, label='With slope adjustment')
plt.plot(xc,qb_noslope, label='No slope adjustment')
plt.plot(xc,zc/100000., label='bed')
plt.legend(loc=1)
plt.ylabel('Bedload kg/s')

In [ ]:
slopZ0 = np.gradient(znew)
plt.plot(xc,slopZ0)
plt.plot(xc,zc)

In [ ]:
min(slopZ),max(slopZ)